In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def algoritmo_euclides():
  a = np.random.randint(1, 101)
  b = np.random.randint(1, 101)
  return a, b, math.gcd(a, b)

def create_data(n):
  data = []

  while len(data) < n:
    a, b, mdc = algoritmo_euclides()
    data.append([a, b, mdc])

  # Escreve todos os dados em um arquivo
  with open("data.txt", "w") as f:
    for row in data:
      f.write("\t".join(map(str, row)) + "\n")

def load_data(file_path):
  with open(file_path, "r") as f:
    data = [line.strip().split("\t") for line in f.readlines()]

  y_in = np.array([[int(d[0]), int(d[1])] for d in data])
  mdc = np.array([int(d[2]) for d in data])

  # Primeira divisão entre treinamento e teste
  y_train, y_test, mdc_train, mdc_test = train_test_split(y_in, mdc, test_size=0.5, shuffle=False, random_state=42)
  # Segunda divisão dentro do conjunto de teste, entre teste e validação
  y_test, y_validation, mdc_test, mdc_validation = train_test_split(y_test, mdc_test, test_size=0.5, shuffle=False, random_state=42)

  return y_train, y_test, y_validation, mdc_train, mdc_test, mdc_validation

In [ ]:
n = 2000000
create_data(n)

y_train, y_test, y_validation, mdc_train, mdc_test, mdc_validation = load_data("data.txt")

In [ ]:
y_train, y_test, y_validation, mdc_train, mdc_test, mdc_validation = load_data("data.txt")

In [ ]:
#Rede Neural
model = Sequential([
  Embedding(input_dim=101, output_dim=64), # Input Layer
  LSTM(64, return_sequences=True),
  LSTM(64, return_sequences=True),
  LSTM(64),
  Dense(1, activation="linear") # Output layer
])

# Compilar o modelo
model.compile(optimizer = "adam", loss= "mean_squared_error", metrics=["accuracy"])

# Callback para parar o treinamento se a métrica monitorada não melhorar
early_stopping = EarlyStopping(monitor="loss", patience=10)

#Treina o modelo
history = model.fit(y_train, mdc_train, epochs=100, batch_size=400, verbose=1, callbacks=[early_stopping], validation_data=(y_validation, mdc_validation))

In [ ]:
# Criar a figura e os subplots
fig, axs = plt.subplots(1,2, figsize=(14, 6))

# Plotar o gráfico da CUSTO em função da época
axs[0].plot(history.history['loss'], label="Custo (Dados de treinamento)")
axs[0].plot(history.history['val_loss'], label="Custo (Dados de teste)")
axs[0].set_title('Custo em função da Época')
axs[0].set_xlabel('Época')
axs[0].set_ylabel('Custo')
axs[0].legend(loc="upper right")
axs[0].grid(True)

# Plotar o gráfico da PRECISÃO em função da época
axs[1].plot(history.history['accuracy'], label="Precisão (Dados de treinamento)")
axs[1].plot(history.history['val_accuracy'], label="Precisão (Dados de teste)")
axs[1].set_title('Precisão em função da Época')
axs[1].set_xlabel('Época')
axs[1].set_ylabel('Precisão')
axs[1].legend(loc="lower right")
axs[1].grid(True)

# Ajustar o layout
plt.tight_layout()

# Mostrar a figura
plt.show()

In [ ]:
predictions = model.predict(y_validation)

# Precisões
precisions = []

menor = []
maior = []
entre = []

for i in range(len(y_validation)):
  real_value = mdc_validation[i]  # Valor real esperado
  predicted_value = predictions[i][0] # Valor previsto pela rede 
  precisions.append( (predicted_value / real_value) * 100 )

# Separa as precisões nas respectivas listas
for i in range(len(precisions)):
  if precisions[i] > 100:
    maior.append(precisions[i])
  elif precisions[i] < 90:
    menor.append(precisions[i])
  else:
    entre.append(precisions[i])

print(f"Dados (Validação): {len(precisions)}")
print(f"Dados com precisão entre 90% e 100%: {len(entre)}")
print(f"Dados com precisão menor que 90%: {len(menor)}")
print(f"Dados com precisão maior que 100%: {len(maior)}")

print(f"MENOR: {min(precisions)}%")
print(f"MAIOR: {max(precisions)}%")

# Mostra as precisões
for i in range(len(precisions)):
  print(f"\nMDC de {y_validation[i][0], y_validation[i][1]} - Valor real: {mdc_validation[i]} - Valor calculado: {predictions[i][0]} - Precisão: {precisions[i]}%")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Obtém os pesos e bias da camada de Embedding
embedding_weights = model.layers[0].get_weights()[0]

# Obtém os pesos e bias da primeira camada LSTM
lstm1_weights, lstm1_recurrent_weights, lstm1_bias = model.layers[1].get_weights()

# Obtém os pesos e bias da segunda camada LSTM
lstm2_weights, lstm2_recurrent_weights, lstm2_bias = model.layers[2].get_weights()

# Obtém os pesos e bias da terceira camada LSTM
lstm3_weights, lstm3_recurrent_weights, lstm3_bias = model.layers[3].get_weights()

# Obtém os pesos e bias da camada densa
dense_weights, dense_bias = model.layers[4].get_weights()

# Função para plotar heatmaps em subplots
def plot_weights_and_biases(weights, bias, title_weights, title_bias):
  plt.figure(figsize=(14, 6))
    
  # Plot weights
  plt.subplot(1, 2, 1)
  sns.heatmap(weights, cmap="viridis", linewidths=.5)
  plt.title(title_weights)
    
  # Plot bias
  plt.subplot(1, 2, 2)
  sns.heatmap(bias.reshape(1, -1), cmap="viridis", linewidths=.5)
  plt.title(title_bias)
    
  plt.tight_layout()
  plt.show()

# Mapa de calor dos pesos e bias da camada de Embedding
plot_weights_and_biases(embedding_weights, embedding_weights.mean(axis=0), 'Embedding Layer Weights', 'Embedding Layer Bias')

# Mapa de calor dos pesos e bias da primeira camada LSTM
plot_weights_and_biases(lstm1_weights, lstm1_bias, 'LSTM 1 Weights', 'LSTM 1 Bias')

# Mapa de calor dos pesos e bias da segunda camada LSTM
plot_weights_and_biases(lstm2_weights, lstm2_bias, 'LSTM 2 Weights', 'LSTM 2 Bias')

# Mapa de calor dos pesos e bias da terceira camada LSTM
plot_weights_and_biases(lstm3_weights, lstm3_bias, 'LSTM 3 Weights', 'LSTM 3 Bias')

# Mapa de calor dos pesos e bias da camada densa
plot_weights_and_biases(dense_weights, dense_bias, 'Dense Layer Weights', 'Dense Layer Bias')
